In [0]:
%pip install -U -qqqq pydantic
%restart_python

In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
import sys, os, yaml
sys.path.append(os.path.abspath('..'))
from configs.project import ProjectConfig

with open("../configs/project.yml", "r") as file:
    data = yaml.safe_load(file)

projectConfig = ProjectConfig(**data)

In [0]:
_config = projectConfig.vector_search_attributes["id_1"]

In [0]:
import urllib.request

_ = urllib.request.urlretrieve(_config.url, _config.local_path)

In [0]:
spark.read.parquet(_config.local_path).write.format("delta").mode("overwrite").saveAsTable(
    _config.source_table_name, mode="overwrite",
)

In [0]:
# spark.sql(f"ALTER TABLE {_config.source_table_name} ALTER COLUMN {_config.primary_key} SET NOT NULL")
# try:
#   spark.sql(f"ALTER TABLE {_config.source_table_name} ADD CONSTRAINT {_config.primary_key}_pk PRIMARY KEY( {_config.primary_key} )")
# except Exception as e:
#   print(f"Constraint {_config.primary_key}_pk already exists.")
# spark.sql(f"ALTER TABLE {_config.source_table_name} SET TBLPROPERTIES (delta.enableChangeDataFeed = true) ")